<a href="https://colab.research.google.com/github/davidharrisnet/genai/blob/main/Copy_of_PE_llama2_13b_chat_hf_questions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#https://www.kaggle.com/code/gpreda/rag-using-llama-2-langchain-and-chromadb

In [ ]:
!pip install transformers==4.33.0 accelerate==0.22.0 einops==0.6.1 langchain==0.0.300 xformers==0.0.21 \
bitsandbytes==0.41.1 sentence_transformers==2.2.2 chromadb==0.4.12 --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 62.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.0/167.0 MB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 10.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 426.5/426.5 kB 42.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 116.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 80.4 MB/s e

In [ ]:
from torch import cuda, bfloat16
import torch
import transformers
from transformers import AutoTokenizer
from time import time
#import chromadb
#from chromadb.config import Settings
from langchain.llms import HuggingFacePipeline
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
cuda.is_available()

True

In [ ]:
model_id = os.path.join("/content","drive","My Drive", "models", "Llama-2-13b-chat-hf" , "models--meta-llama--Llama-2-13b-chat-hf", "snapshots", "1")
os.path.exists(model_id)

True

In [ ]:
#model_id = os.path.join("/content","drive","My Drive", "models", "Llama-2-7b-hf", "snapshots", "1")

In [ ]:
#os.path.exists(model_id)


In [ ]:

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

# set quantization configuration to load large model with less GPU memory
# this requires the `bitsandbytes` library
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)

In [ ]:
time_1 = time()
model_config = transformers.AutoConfig.from_pretrained(
    model_id,
)
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto',
)
tokenizer = AutoTokenizer.from_pretrained(model_id)
time_2 = time()
print(f"Prepare model, tokenizer: {round(time_2-time_1, 3)} sec.")

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Prepare model, tokenizer: 277.226 sec.


In [ ]:


time_1 = time()
query_pipeline = transformers.pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        torch_dtype=torch.float16,
        device_map="auto",)
time_2 = time()
print(f"Prepare pipeline: {round(time_2-time_1, 3)} sec.")



Prepare pipeline: 0.889 sec.


In [ ]:
questions = [ "Write me a detailed scenario for a potential Russian invasion of Moldova.",

"Now identify one person in this scenario who would be a potential HUMINT source for the US Intelligence Community and write a short source dossier. Be sure to include their placement and access.",
"Write me three short intelligence information reports that came from that source about various events in the scenario.",
"Add dates and times for the intelligence reports." ]




In [ ]:
!pip install langchain[all]==0.0.300 --quiet

In [ ]:
from langchain.llms import HuggingFacePipeline
from transformers import AutoTokenizer
from langchain.chains import ConversationChain
from langchain import PromptTemplate
from langchain.memory import ConversationBufferWindowMemory
from langchain.chains.conversation.memory import  ConversationSummaryMemory


In [ ]:
pipeline=transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
    max_length=10000,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id
    )

llm=HuggingFacePipeline(pipeline=pipeline, model_kwargs={'temperature':0.7})

In [ ]:
prompt_template = """<s>[INST] <<SYS>>
{{ You are an AP reporter.}}<<SYS>>
###

Previous Conversation:
'''
{history}
'''

{{{input}}}[/INST]

"""
prompt  = PromptTemplate(template=prompt_template, input_variables=['input', 'history'])

In [ ]:
#memory = ConversationSummaryMemory(llm=llm)
memory = ConversationBufferWindowMemory(k=10)

chain = ConversationChain(
    llm=llm,
    prompt=prompt,
    memory=memory
)

In [ ]:
chain.run(questions[0])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


'As an AP reporter, I have been monitoring the situation in Moldova closely, and I have developed a potential scenario for a Russian invasion. Here is my analysis:\n\nScenario: Russian Invasion of Moldova\n\nBackground: Tensions between Russia and Moldova have been escalating in recent months, with Russia accusing Moldova of harboring pro-Western and anti-Russian sentiments. The situation has been further complicated by the ongoing conflict in Ukraine, which has created a power vacuum in the region that Russia is eager to fill.\n\nInciting Incident: In the early hours of the morning, Russian forces launch a surprise invasion of Moldova, using a combination of airborne troops, armor, and special operations forces to quickly overrun key strategic points in the country. The initial attack focuses on the capital city of Chisinau, where key government buildings and infrastructure are targeted.\n\nDevelopment: The initial attack is swift and brutal, with Russian forces quickly gaining contro

In [ ]:
chain.run(questions[1])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


"Name: Viktor Kuznetsov\n\nAge: 42\n\nNationality: Moldovan\n\nPlacement: Governor of the Chisinau Province\n\nAccess: Viktor has access to high-level government meetings and conversations, as well as to sensitive information about the Moldovan military's capabilities and preparedness. He is also well-connected within the pro-Russian community in Moldova, and has a network of sources that can provide information on Russian activities in the country.\n\nBackground: Viktor is a former member of the Moldovan military who was appointed as the governor of the Chisinau Province by the current pro-Russian government. He is a controversial figure, known for his pro-Russian stance and his efforts to weaken the Moldovan military's ties to the West. Despite his background, Viktor is well-liked by many in the region for his charismatic leadership style and his ability to get things done.\n\nReason for Recruitment: With the Russian invasion of Moldova underway, the US Intelligence Community is in u

In [ ]:
chain.run(questions[2])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


"Report 1:\n\nSubject: Russian Military Movements in Eastern Moldova\n\nDate: March 22, 2023\n\nLocation: Eastern Moldova\n\nIntelligence Gathered: On March 22, 2023, a HUMINT source in eastern Moldova reported that Russian military convoys have been moving towards the city of Tighina, a strategic location near the breakaway region of Transnistria. The convoys are believed to be carrying troops and equipment for a potential assault on the city. The source also reported that Russian-backed separatist forces in Transnistria have been mobilizing and fortifying their positions.\n\nAnalysis: The Russian military's movements in eastern Moldova could be indicative of a potential invasion. The targeting of Tighina, with its strategic location near Transnistria, could be an attempt by Russia to gain a foothold in the region and secure its interests.\n\nReport 2:\n\nSubject: Pro-Russian Protests in Chisinau\n\nDate: March 23, 2023\n\nLocation: Chisinau, Moldova\n\nIntelligence Gathered: On March

In [ ]:
chain.run(questions[3])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


"Report 1:\n\nSubject: Russian Military Movements in Eastern Moldova\n\nDate: March 22, 2023, 10:00 AM\n\nLocation: Eastern Moldova\n\nIntelligence Gathered: On March 22, 2023, at 10:00 AM, a HUMINT source in eastern Moldova reported that Russian military convoys have been moving towards the city of Tighina, a strategic location near the breakaway region of Transnistria. The convoys are believed to be carrying troops and equipment for a potential assault on the city. The source also reported that Russian-backed separatist forces in Transnistria have been mobilizing and fortifying their positions.\n\nAnalysis: The Russian military's movements in eastern Moldova could be indicative of a potential invasion. The targeting of Tighina, with its strategic location near Transnistria, could be an attempt by Russia to gain a foothold in the region and secure its interests.\n\nReport 2:\n\nSubject: Pro-Russian Protests in Chisinau\n\nDate: March 23, 2023, 3:00 PM\n\nLocation: Chisinau, Moldova\n\